# Tutorial

## Tutorial

### Setting up environment

In [ ]:
  import sys
  import os
  import numpy as np
  import pandas as pd
  import scanpy as sc
  import matplotlib.pyplot as plt
  import nsforest as ns
  from nsforest import utils
  from nsforest import preprocessing as pp
  from nsforest import nsforesting
  from nsforest import evaluating as ev
  from nsforest import plotting as pl#+end_src

*** Data Exploration

**** Loading h5ad AnnData file

#+begin_src jupyter-python
  data_folder = "../data/cellxgene-test/"
  file = data_folder + "adata_layer1.h5ad"
  adata = sc.read_h5ad(file)
  adata#+end_src

**** Defining ~cluster_header~ as cell type annotation. 

*Note:* Some datasets have multiple annotations per sample (ex. "broad_cell_type" and "granular_cell_type"). NS-Forest can be run on multiple ~cluster_header~'s. Combining the parent and child markers may improve classification results. 

#+begin_src jupyter-python
  cluster_header = "cluster"#+end_src

**** Defining ~output_folder~ for saving results

#+begin_src jupyter-python
  output_folder = "results-from-tutorail/"#+end_src

**** Looking at sample labels

#+begin_src jupyter-python
  adata.obs_names#+end_src

**** Looking at genes

*Note:* ~adata.var_names~ must be unique. If there is a problem, usually it can be solved by assigning ~adata.var.index = adata.var["ensembl_id"]~. 

#+begin_src jupyter-python
  adata.var_names#+end_src

**** Checking cell annotation sizes 

*Note:* Some datasets are too large and need to be downsampled to be run through the pipeline. When downsampling, be sure to have all the granular cluster annotations represented. 

#+begin_src jupyter-python
  adata.obs[cluster_header].value_counts()#+end_src

*** Preprocessing

**** Generating scanpy dendrogram

*Note:* Only run if there is no pre-defined dendrogram order. This step can still be run with no effects, but the runtime may increase. 

Dendrogram order is stored in ~adata.uns["dendrogram_cluster"]["categories_ordered"]~. 

This dataset has a pre-defined dendrogram order, so running this step is not necessary. 

#+begin_src jupyter-python
  ns.pp.dendrogram(adata, cluster_header, save = True, output_folder = output_folder, outputfilename_suffix = cluster_header)#+end_src

**** Calculating cluster medians per gene

Run ~ns.pp.prep_medians~ before running NS-Forest.

*Note:* Do *not* run if evaluating marker lists. Do *not* run when generating scanpy plots (e.g. dot plot, violin plot, matrix plot). 

#+begin_src jupyter-python
  adata = ns.pp.prep_medians(adata, cluster_header)
  adata#+end_src

**** Calculating binary scores per gene per cluster

Run ~ns.pp.prep_binary_scores~ before running NS-Forest. Do not need to run if evaluating marker lists. Do not need to run when generating scanpy plots. 

#+begin_src jupyter-python
  adata = ns.pp.prep_binary_scores(adata, cluster_header)
  adata#+end_src

**** Plotting median and binary score distributions

#+begin_src jupyter-python
  plt.clf()
  filename = output_folder + cluster_header + '_medians.png'
  print(f"Saving median distributions as...\n{filename}")
  a = plt.figure(figsize = (6, 4))
  a = plt.hist(adata.varm["medians_" + cluster_header].unstack(), bins = 100)
  a = plt.title(f'{file.split("/")[-1].replace(".h5ad", "")}: {"medians_" + cluster_header} histogram')
  a = plt.xlabel("medians_" + cluster_header)
  a = plt.yscale("log")
  a = plt.savefig(filename, bbox_inches='tight')
  plt.show()#+end_src

#+begin_src jupyter-python
  plt.clf()
  filename = output_folder + cluster_header + '_binary_scores.png'
  print(f"Saving binary_score distributions as...\n{filename}")
  a = plt.figure(figsize = (6, 4))
  a = plt.hist(adata.varm["binary_scores_" + cluster_header].unstack(), bins = 100)
  a = plt.title(f'{file.split("/")[-1].replace(".h5ad", "")}: {"binary_scores_" + cluster_header} histogram')
  a = plt.xlabel("binary_scores_" + cluster_header)
  a = plt.yscale("log")
  a = plt.savefig(filename, bbox_inches='tight')
  plt.show()#+end_src

**** Saving preprocessed AnnData as new h5ad

#+begin_src jupyter-python
  filename = file.replace(".h5ad", "_preprocessed.h5ad")
  print(f"Saving new anndata object as...\n{filename}")
  adata.write_h5ad(filename)#+end_src

*** Running NS-Forest

*Note:* Do not run NS-Forest if only evaluating input marker lists. 

#+begin_src jupyter-python :scrolled True
  outputfilename_prefix = cluster_header
  results = nsforesting.NSForest(adata, cluster_header, output_folder = output_folder, outputfilename_prefix = outputfilename_prefix)#+end_src

#+begin_src jupyter-python :scrolled True
  results#+end_src

**** Plotting classification metrics from NS-Forest results

#+begin_src jupyter-python
  ns.pl.boxplot(results, "f_score")#+end_src

#+begin_src jupyter-python
  ns.pl.boxplot(results, "PPV")#+end_src

#+begin_src jupyter-python
  ns.pl.boxplot(results, "recall")#+end_src

#+begin_src jupyter-python
  ns.pl.boxplot(results, "onTarget")#+end_src

#+begin_src jupyter-python
  ns.pl.scatter_w_clusterSize(results, "f_score")#+end_src

#+begin_src jupyter-python
  ns.pl.scatter_w_clusterSize(results, "PPV")#+end_src

#+begin_src jupyter-python
  ns.pl.scatter_w_clusterSize(results, "recall")#+end_src

#+begin_src jupyter-python
  ns.pl.scatter_w_clusterSize(results, "onTarget")#+end_src

*** Plotting scanpy dot plot, violin plot, matrix plot for NS-Forest markers

*Note:* Assign pre-defined dendrogram order here *or* use ~adata.uns["dendrogram_" + cluster_header]["categories_ordered"]~. 

#+begin_src jupyter-python
  to_plot = results.copy()#+end_src

#+begin_src jupyter-python
  dendrogram = [] # custom dendrogram order
  dendrogram = list(adata.uns["dendrogram_" + cluster_header]["categories_ordered"])
  to_plot["clusterName"] = to_plot["clusterName"].astype("category")
  to_plot["clusterName"] = to_plot["clusterName"].cat.set_categories(dendrogram)
  to_plot = to_plot.sort_values("clusterName")
  to_plot = to_plot.rename(columns = {"NSForest_markers": "markers"})
  to_plot.head()#+end_src

#+begin_src jupyter-python
  markers_dict = dict(zip(to_plot["clusterName"], to_plot["markers"]))
  markers_dict#+end_src

#+begin_src jupyter-python
  ns.pl.dotplot(adata, markers_dict, cluster_header, dendrogram = dendrogram, save = True, output_folder = output_folder, outputfilename_suffix = outputfilename_prefix)#+end_src

#+begin_src jupyter-python
  ns.pl.stackedviolin(adata, markers_dict, cluster_header, dendrogram = dendrogram, save = True, output_folder = output_folder, outputfilename_suffix = outputfilename_prefix)#+end_src

#+begin_src jupyter-python
  ns.pl.matrixplot(adata, markers_dict, cluster_header, dendrogram = dendrogram, save = True, output_folder = output_folder, outputfilename_suffix = outputfilename_prefix)#+end_src

*** Evaluating input marker list

**** Getting marker list in dictionary format: {cluster: marker_list}

#+begin_src jupyter-python :scrolled True
  markers = pd.read_csv("../../NSForest/demo_data/marker_list.csv")
  markers_dict = utils.prepare_markers(markers, "clusterName", "markers")
  markers_dict#+end_src

#+begin_src jupyter-python :scrolled True
  outputfilename_prefix = "marker_eval"
  evaluation_results = ev.DecisionTree(adata, cluster_header, markers_dict, combinations = False, use_mean = False, 
                                               output_folder = output_folder, outputfilename_prefix = outputfilename_prefix)#+end_src

#+begin_src jupyter-python :scrolled True
  evaluation_results#+end_src

**** Plotting classification metrics from marker evaluation

#+begin_src jupyter-python
  ns.pl.boxplot(evaluation_results, "f_score")#+end_src

#+begin_src jupyter-python
  ns.pl.boxplot(evaluation_results, "PPV")#+end_src

#+begin_src jupyter-python
  ns.pl.boxplot(evaluation_results, "recall")#+end_src

#+begin_src jupyter-python
  ns.pl.boxplot(evaluation_results, "onTarget")#+end_src

#+begin_src jupyter-python
  ns.pl.scatter_w_clusterSize(evaluation_results, "f_score")#+end_src

#+begin_src jupyter-python
  ns.pl.scatter_w_clusterSize(evaluation_results, "PPV")#+end_src

#+begin_src jupyter-python
  ns.pl.scatter_w_clusterSize(evaluation_results, "recall")#+end_src

#+begin_src jupyter-python
  ns.pl.scatter_w_clusterSize(evaluation_results, "onTarget")#+end_src

*** Plotting scanpy dot plot, violin plot, matrix plot for input marker list

*Note:* Assign pre-defined dendrogram order here *or* use ~adata.uns["dendrogram_" + cluster_header]["categories_ordered"]~. 

#+begin_src jupyter-python
  to_plot = evaluation_results.copy()#+end_src

#+begin_src jupyter-python
  dendrogram = [] # custom dendrogram order
  dendrogram = list(adata.uns["dendrogram_" + cluster_header]["categories_ordered"])
  to_plot["clusterName"] = to_plot["clusterName"].astype("category")
  to_plot["clusterName"] = to_plot["clusterName"].cat.set_categories(dendrogram)
  to_plot = to_plot.sort_values("clusterName")
  to_plot = to_plot.rename(columns = {"NSForest_markers": "markers"})
  to_plot.head()#+end_src

#+begin_src jupyter-python
  markers_dict = dict(zip(to_plot["clusterName"], to_plot["markers"]))
  markers_dict#+end_src

#+begin_src jupyter-python
  ns.pl.dotplot(adata, markers_dict, cluster_header, dendrogram = dendrogram, save = True, output_folder = output_folder, outputfilename_suffix = outputfilename_prefix)#+end_src

#+begin_src jupyter-python
  ns.pl.stackedviolin(adata, markers_dict, cluster_header, dendrogram = dendrogram, save = True, output_folder = output_folder, outputfilename_suffix = outputfilename_prefix)#+end_src

#+begin_src jupyter-python
  ns.pl.matrixplot(adata, markers_dict, cluster_header, dendrogram = dendrogram, save = True, output_folder = output_folder, outputfilename_suffix = outputfilename_prefix)#+end_src

#+begin_src jupyter-python
